# Combinación de ambos modelos

En este notebook vamos a recrear el proceso completo con los dos modelos que hemos entrenado, desde cargar una imagen inicial hasta la obtención del archivo MIDI final.

## Cargamos los modelos

In [1]:
from model.extender_midi.model import MelodIAModel, Utilities
from model.foto_a_midi.main import FotoAMIDI

# Cargamos los modelos
MODEL_PARAMS_PATH = "pretrained_models/modelo_params_midi.h5"
MODEL_MIDI_EXTENSION_PATH = "pretrained_models/modelo_extension_midi.keras"

modelo_params = FotoAMIDI(modelo_ruta=MODEL_PARAMS_PATH, usar_clip=True)
modelo_params.load_modelo()
modelo_extender_midi = MelodIAModel.load_model(MODEL_MIDI_EXTENSION_PATH)

2025-05-18 18:04:57.631376: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-05-18 18:04:57.631442: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-18 18:04:57.631463: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-18 18:04:57.631492: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-18 18:04:57.631516: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Modelo cargado desde: pretrained_models/modelo_extension_midi.keras


## Modelo 1: Foto a MIDI

In [2]:
import os
from PIL import Image
from Scripts.python.midi_generator_acordes import ParamsAMIDI
# Cargamos la imagen
IMAGE_PATH = "./model/foto_a_midi/datos/test/noche.jpeg"
im = Image.open(IMAGE_PATH)


In [3]:
# Metemos la imagen en el modelo y predecimos sus parametros
params = modelo_params.predict(im)


: 

In [ ]:
MIDI_PATH = "prueba.mid"
# Generamos el midi a partir de los parámetros obtenidos
midi_basico = ParamsAMIDI(**params)
midi_basico.generar_midi(MIDI_PATH)

# Si el midi tiene menos de 25 notas, lo concatenamos
import pretty_midi

pm = pretty_midi.PrettyMIDI(MIDI_PATH)
num_notas = len(pm.instruments[0].notes)
if (num_notas < 25):
    n = 25 / num_notas + 1
    aux = pm
    for i in range(0, n):
        aux = Utilities.concatenate_pretty_midi(pm, aux)
    pm = aux

## Modelo 2: Extensión del archivo MIDI generado

In [ ]:
import pretty_midi

df = Utilities.midi_path_to_df(MIDI_PATH)
notes = Utilities.midi_to_numpy(pm)
notes_sequence = notes[:25] # primeras 25 notas
notes = MelodIAModel.normalize(notes_sequence) 

Generamos las nuevas notas con el modelo

In [ ]:
# Se pueden cambiar el número de predicciones y la temperatura
generated_notes = MelodIAModel.predict_sequence(modelo_extender_midi, notes, num_predictions=25, temperature=0.8)

generated_notes = MelodIAModel.desnormalize(generated_notes)
generado = Utilities.notes_to_midi(generated_notes)

Unimos el fragmento MIDI original con el generado

In [ ]:
original = Utilities.notes_to_midi(notes_sequence)
combined = Utilities.concatenate_pretty_midi(original, generado)

Lo visualizamos

In [ ]:
Utilities.plot_piano_roll(combined, 40, 100)

También se puede escuchar

In [ ]:
Utilities.display_audio(combined)

Por último lo guardamos

In [ ]:
combined.write("midi_final.mid")